# Vytvořte klasifikační model: Lahodné asijské a indické kuchyně


## Klasifikátory kuchyní 2

V této druhé lekci o klasifikaci se podíváme na `další způsoby`, jak klasifikovat kategorická data. Také se zaměříme na důsledky výběru jednoho klasifikátoru oproti jinému.

### [**Kvíz před lekcí**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Předpoklady**

Předpokládáme, že jste dokončili předchozí lekce, protože budeme navazovat na některé koncepty, které jsme se již naučili.

Pro tuto lekci budeme potřebovat následující balíčky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekce balíčků pro R](https://www.tidyverse.org/packages), která usnadňuje, zrychluje a zpříjemňuje práci s datovou vědou!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je rámec [kolekce balíčků](https://www.tidymodels.org/packages/) pro modelování a strojové učení.

-   `themis`: [balíček themis](https://themis.tidymodels.org/) poskytuje další kroky pro recepty, které řeší problém nevyvážených dat.

Můžete je nainstalovat pomocí:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativně níže uvedený skript zkontroluje, zda máte balíčky potřebné k dokončení tohoto modulu, a v případě, že chybí, je nainstaluje.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Mapa klasifikace**

V naší [předchozí lekci](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) jsme se snažili odpovědět na otázku: jak si vybrat mezi více modely? Do značné míry to závisí na charakteristikách dat a typu problému, který chceme řešit (například klasifikace nebo regrese?).

Dříve jsme se seznámili s různými možnostmi, které máte při klasifikaci dat, pomocí přehledové tabulky od Microsoftu. Pythonův framework pro strojové učení, Scikit-learn, nabízí podobnou, ale podrobnější přehledovou tabulku, která vám může dále pomoci zúžit výběr odhadovačů (jiný termín pro klasifikátory):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tip: [navštivte tuto mapu online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) a klikněte na cestu, abyste si přečetli dokumentaci.
>
> [Referenční stránka Tidymodels](https://www.tidymodels.org/find/parsnip/#models) také poskytuje vynikající dokumentaci o různých typech modelů.

### **Plán** 🗺️

Tato mapa je velmi užitečná, jakmile máte jasnou představu o svých datech, protože se můžete „procházet“ po jejích cestách k rozhodnutí:

-   Máme více než 50 vzorků

-   Chceme předpovědět kategorii

-   Máme označená data

-   Máme méně než 100 tisíc vzorků

-   ✨ Můžeme zvolit Linear SVC

-   Pokud to nefunguje, protože máme číselná data

    -   Můžeme zkusit ✨ KNeighbors Classifier

        -   Pokud to nefunguje, zkusíme ✨ SVC a ✨ Ensemble Classifiers

Toto je velmi užitečná cesta, kterou se můžeme řídit. Teď se do toho pustíme pomocí [tidymodels](https://www.tidymodels.org/) frameworku pro modelování: konzistentní a flexibilní kolekce balíčků v R, vyvinutá pro podporu dobré statistické praxe 😊.

## 2. Rozdělení dat a práce s nevyváženým datovým souborem.

Z našich předchozích lekcí jsme se naučili, že existuje sada společných ingrediencí napříč našimi kuchyněmi. Také jsme zjistili, že rozložení počtu kuchyní bylo dost nerovnoměrné.

S tím se vypořádáme takto:

-   Odstraníme nejběžnější ingredience, které vytvářejí zmatek mezi odlišnými kuchyněmi, pomocí `dplyr::select()`.

-   Použijeme `recipe`, který předzpracuje data, aby byla připravena pro modelování, aplikací algoritmu `over-sampling`.

Toto jsme již probírali v předchozí lekci, takže to bude hračka 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Práce s nevyváženými daty

Nevyvážená data často negativně ovlivňují výkon modelu. Mnoho modelů dosahuje nejlepších výsledků, když je počet pozorování vyrovnaný, a proto mají tendenci mít problémy s nevyváženými daty.

Existují dvě hlavní metody, jak pracovat s nevyváženými datovými sadami:

-   přidání pozorování do menšinové třídy: `Over-sampling`, například použitím algoritmu SMOTE, který synteticky generuje nové příklady menšinové třídy na základě nejbližších sousedů těchto případů.

-   odstranění pozorování z většinové třídy: `Under-sampling`

V naší předchozí lekci jsme ukázali, jak pracovat s nevyváženými datovými sadami pomocí `receptu`. Recept si můžeme představit jako plán, který popisuje, jaké kroky by měly být aplikovány na datovou sadu, aby byla připravena pro analýzu dat. V našem případě chceme dosáhnout rovnoměrného rozložení počtu našich kuchyní v `trénovací sadě`. Pojďme se do toho pustit.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Teď jsme připraveni trénovat modely 👩‍💻👨‍💻!

## 3. Nad rámec multinomiálních regresních modelů

V naší předchozí lekci jsme se zabývali multinomiálními regresními modely. Pojďme prozkoumat některé flexibilnější modely pro klasifikaci.

### Support Vector Machines

V kontextu klasifikace je `Support Vector Machines` technika strojového učení, která se snaží najít *hyperrovinu*, která "nejlépe" odděluje třídy. Podívejme se na jednoduchý příklad:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ nerozděluje třídy. H2~ je rozděluje, ale pouze s malým okrajem. H3~ je rozděluje s maximálním okrajem.

#### Lineární klasifikátor podpůrných vektorů

Shlukování pomocí podpůrných vektorů (SVC) je součástí rodiny technik strojového učení založených na podpůrných vektorech. V SVC je hyperrovina zvolena tak, aby správně oddělila `většinu` tréninkových pozorování, ale `může nesprávně klasifikovat` několik pozorování. Tím, že umožníme některým bodům být na nesprávné straně, se SVM stává odolnějším vůči odlehlým hodnotám, což vede k lepší generalizaci na nová data. Parametr, který reguluje toto porušení, se nazývá `cost` a má výchozí hodnotu 1 (viz `help("svm_poly")`).

Vytvořme lineární SVC nastavením `degree = 1` v polynomiálním modelu SVM.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nyní, když jsme zachytili kroky předzpracování a specifikaci modelu do *workflow*, můžeme pokračovat s trénováním lineárního SVC a zároveň vyhodnocovat výsledky. Pro metriky výkonu vytvořme sadu metrik, která bude hodnotit: `přesnost`, `citlivost`, `Pozitivní prediktivní hodnotu` a `F míru`.

> `augment()` přidá sloupec/sloupce s predikcemi do daných dat.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Support Vector Machine (SVM) je rozšířením support vector classifier, které umožňuje zohlednit nelineární hranici mezi třídami. V podstatě SVM využívají *kernel trick* k rozšíření prostoru příznaků, aby se přizpůsobily nelineárním vztahům mezi třídami. Jednou z populárních a velmi flexibilních kernel funkcí používaných v SVM je *Radial basis function.* Podívejme se, jak si povede na našich datech.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mnohem lepší 🤩!

> ✅ Podívejte se:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> pro další čtení.

### Klasifikátory nejbližšího souseda

*K*-nejbližší soused (KNN) je algoritmus, ve kterém je každé pozorování předpovězeno na základě jeho *podobnosti* s ostatními pozorováními.

Pojďme ho aplikovat na naše data.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Zdá se, že tento model nefunguje příliš dobře. Pravděpodobně změna parametrů modelu (viz `help("nearest_neighbor")`) zlepší jeho výkon. Určitě to vyzkoušejte.

> ✅ Podívejte se:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> pro více informací o klasifikátorech *K*-Nearest Neighbors.

### Ensemble klasifikátory

Ensemble algoritmy fungují tak, že kombinují více základních odhadů, aby vytvořily optimální model buď:

`bagging`: použitím *průměrovací funkce* na kolekci základních modelů

`boosting`: vytvořením sekvence modelů, které na sebe navazují a zlepšují prediktivní výkon.

Začněme tím, že vyzkoušíme model Random Forest, který vytváří velkou kolekci rozhodovacích stromů a poté aplikuje průměrovací funkci pro vytvoření lepšího celkového modelu.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Dobrá práce 👏!

Pojďme také experimentovat s modelem Boosted Tree.

Boosted Tree definuje ensemble metodu, která vytváří sérii sekvenčních rozhodovacích stromů, kde každý strom závisí na výsledcích předchozích stromů ve snaze postupně snižovat chybu. Zaměřuje se na váhy nesprávně klasifikovaných položek a upravuje přizpůsobení pro další klasifikátor, aby chybu napravil.

Existují různé způsoby, jak tento model přizpůsobit (viz `help("boost_tree")`). V tomto příkladu budeme přizpůsobovat Boosted trees pomocí enginu `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Podívejte se:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Zkoumá model AdaBoost, který je dobrou alternativou k xgboost.
>
> pro více informací o Ensemble klasifikátorech.

## 4. Extra - porovnání více modelů

V tomto cvičení jsme vytvořili poměrně velké množství modelů 🙌. Může být únavné nebo zdlouhavé vytvářet mnoho workflowů z různých sad předzpracování a/nebo specifikací modelů a poté jeden po druhém počítat metriky výkonu.

Podívejme se, zda to můžeme vyřešit vytvořením funkce, která aplikuje seznam workflowů na trénovací sadu a poté vrátí metriky výkonu na základě testovací sady. Použijeme `map()` a `map_dfr()` z balíčku [purrr](https://purrr.tidyverse.org/) k aplikaci funkcí na každý prvek seznamu.

> Funkce [`map()`](https://purrr.tidyverse.org/reference/map.html) vám umožňují nahradit mnoho for smyček kódem, který je stručnější a snadněji čitelný. Nejlepší místo, kde se naučit o funkcích [`map()`](https://purrr.tidyverse.org/reference/map.html), je [kapitola o iteraci](http://r4ds.had.co.nz/iteration.html) v R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) balíček umožňuje uživatelům vytvářet a snadno aplikovat velké množství modelů, ale je primárně navržen pro práci s technikami resamplingu, jako je `cross-validation`, přístup, který teprve budeme probírat.

## **🚀Výzva**

Každá z těchto technik má velké množství parametrů, které můžete upravovat, například `cost` u SVM, `neighbors` u KNN, `mtry` (náhodně vybraní prediktory) u Random Forest.

Prozkoumejte výchozí parametry každého z nich a zamyslete se nad tím, co by úprava těchto parametrů znamenala pro kvalitu modelu.

Chcete-li zjistit více o konkrétním modelu a jeho parametrech, použijte: `help("model")`, například `help("rand_forest")`.

> V praxi obvykle *odhadujeme* *nejlepší hodnoty* těchto parametrů tím, že trénujeme mnoho modelů na `simulovaném datovém souboru` a měříme, jak dobře všechny tyto modely fungují. Tento proces se nazývá **ladění**.

### [**Kvíz po přednášce**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Přehled & Samostudium**

V těchto lekcích je hodně odborných termínů, takže si udělejte chvíli na přezkoumání [tohoto seznamu](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) užitečné terminologie!

#### DĚKUJEME:

[`Allison Horst`](https://twitter.com/allison_horst/) za vytvoření úžasných ilustrací, které činí R přívětivějším a poutavějším. Další ilustrace najdete v její [galerii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) a [Jen Looper](https://www.twitter.com/jenlooper) za vytvoření původní verze tohoto modulu v Pythonu ♥️

Šťastné učení,

[Eric](https://twitter.com/ericntay), Zlatý ambasador Microsoft Learn Student.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>



---

**Upozornění**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o co největší přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za závazný zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
